In [4]:
import datetime as dt

from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *

from cluster_luck_functions.cluster_luck_hitting import *
from cluster_luck_functions.cluster_luck_pitching import *
from cluster_luck_functions.cluster_luck_combined import *

from war_functions.pecota_tables import *
#from war_functions.historical_war_table import *
from war_functions.preseason_war_projections import *
from war_functions.preseason_win_percentage import *

current_year = dt.date.today().year

# Creating names.csv

In [3]:
hitters = pd.read_csv('pecota_data/pecota_hitting.csv')
hitters['name_wo_a'] = hitters.first_name + ' ' + hitters.last_name
hitters = hitters[['bpid', 'name', 'name_wo_a']]

pitchers = pd.read_csv('pecota_data/pecota_pitching.csv')
pitchers['name_wo_a'] = pitchers.first_name + ' ' + pitchers.last_name
pitchers = pitchers[['bpid', 'name', 'name_wo_a']]

names = pd.concat([hitters, pitchers])
for i in [1,2,3,4]:
    names[f'name_alt_{i}'] = ''

names.to_csv('pecota_data/names.csv')

# BOY Active Rosters

In [11]:
retrieve_all_active_rosters()
rosters = load_active_rosters()

# Cluster Luck

In [24]:
%%capture

# Hitting

retrieve_historical_hitting_tables([2022, 2021, 2019, 2018])
historical_hitting_table = load_historical_hitting_tables()
calculate_and_save_hitting_linear_regression(historical_hitting_table)
hitting_reg = load_linear_regression("data/hitting_regression.pickle")
prev_year_hitting_table = retrieve_historical_hitting_tables(2022, file_name = None)
prev_year_hitting_adjustment = calculate_predicted_cluster_luck_run_adjustment_hitting(hitting_reg, prev_year_hitting_table)

In [25]:
%%capture

# Pitching

retrieve_historical_pitching_tables([2022, 2021, 2019, 2018])
historical_pitching_table = load_historical_pitching_tables()
calculate_and_save_pitching_linear_regression(historical_pitching_table)
pitching_reg = load_linear_regression('./data/pitching_regression.pickle')
prev_year_pitching_table = retrieve_historical_pitching_tables(2022, file_name = None)
prev_year_pitching_adjustment = calculate_predicted_cluster_luck_run_adjustment_pitching(pitching_reg, prev_year_pitching_table)

In [27]:
# Merging

final_cluster_luck = merge_cluster_luck_tables(prev_year_hitting_adjustment, prev_year_pitching_adjustment, file_name = 'data/final_cluster_luck.csv')

# WAR Functions

In [28]:
######### NEED TO MAKE GUARDIANS CHANGE ###########

pt = load_combined_pecota_table()
retrieve_previous_year_war_table(2022)
prev_year_war = load_previous_year_war_table()
preseason_projections = calculate_preaseason_war_projections(rosters, pt)
final_war_preseason = calculate_final_preseason_war_change(preseason_projections, prev_year_war, current_year = current_year, file_name = 'data/overall_war_predictions_preseason.csv')
final_preseason_win_percentage = calculate_preseason_win_percentage(final_cluster_luck, final_war_preseason, current_year, file_name = 'data/preseason_projections.csv')